In [2]:
import torch
import numpy as np

In [3]:
class UDFT():
    """
    parameters:
    ----------
    - mask_coordinates : coordinates (in meters) of each apertures of the mask.
    - precision : number of digits to consider when comparing uv coordinates of Fourier components.
    - redundant: Use all triangles in the mask if True. Default is to use non-redundant triangles.

    properties:
    ----------
    - nbap: number of apertures in the mask
    - VAC: Virtual Aperture Coordinates (or just mask coordinates)
    - BLM: Baseline Model. Operator that transform the aperture phase vector into visibility phases.
    - UVC: uv coordinates (in meter) -> used to compute NDFTM

    """
    def __init__(self, mask_coordinates, wavelength, redundant=False):
        self.nbap = mask_coordinates.shape[0] 
        self.VAC = mask_coordinates
        self.wavelength = wavelength

        # Build BLM matrix (mapping from aperture to baselines)
        N = self.nbap
        mask = self.VAC
        p = N * (N-1) // 2
        UVC = np.zeros((p, 2))
        BLM = np.zeros((p, N))
        k = 0
        for i in range(N):
            for j in range(i+1, N):
                UVC[k, 0] = mask[i, 0] - mask[j, 0]
                UVC[k, 1] = mask[i, 1] - mask[j, 1]
                BLM[k, i] += 1.0
                BLM[k, j] -= 1.0
                k += 1
        self.BLM = BLM
        self.UVC = UVC

In [4]:
antennas_coordinates = np.zeros(shape = (30, 2))